<a href="https://colab.research.google.com/github/IDL-SG1-BCI-EEG/main/blob/yuhsun/IDL_BCI_EEG_EDA_STD_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IDL Project: Predicting BCI_EEG signals using Deep Learning**

In [1]:
!pip install wandb --quiet
import os

import torch
import torch.nn as nn
from torchsummary import summary
import torchvision

import gc
from tqdm import tqdm
from PIL import Image
import numpy as np
import pandas as pd
import glob
import wandb
import matplotlib.pyplot as plt
import multiprocessing as mp
import scipy

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Num_Workers: {mp.cpu_count()}')
print("Device: ", DEVICE)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.7/201.7 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.0 MB/s eta 0:00:00
Num_Workers: 2
Device:  cuda


In [2]:
# Get BCI_EEG Data 
!mkdir /content/bci_data
!wget -q https://cmu.box.com/shared/static/dje4whisfwszhe2vvfphvahzr563kp2x.zip --content-disposition --show-progress
!unzip -qo 'BCI_EEG_data.zip' -d '/content/bci_data'

"""
Original Data From:
 *https://figshare.com/articles/online_resource/Shared_data_for_exploring_training_effect_in_42_human_subjects_using_a_noninvasive_sensorimotor_rhythm-based_online_BCI/7959572*
"""

BCI_EEG_data.zip    100%[===================>]   4.84G  20.2MB/s    in 4m 20s  


'\nOriginal Data From:\n *https://figshare.com/articles/online_resource/Shared_data_for_exploring_training_effect_in_42_human_subjects_using_a_noninvasive_sensorimotor_rhythm-based_online_BCI/7959572*\n'

# **EDA**

---
...Each file includes the online results from the BCI experimentation for each run (saved in a cell variable ‘BCI_UseResults’), key parameters for the experiment (saved in a structure ‘Experiment_Parm’), key parameters for the state of the raw EEG signal (saved in a structure ‘Experimental_states’), and the raw EEG signal (saved in a variable ‘output_data’).

The raw EEG signals for experiments one and two are composed of 62 channels of EEG data with a sampling frequency of 100Hz, while data for experiment three contains 64 channels of EEG sampled at 128Hz...

In [3]:
config = {
    'epochs'        : 20,
    'batch_size'    : 32,
    'lr'            : 5e-4, # 0.1 for SGD, 5e-4 for adamW
    'patience'      : 2,
    'lr_decay'      : 0.1,
    'weight_decay'  : 0.001,
    'momentum'      : 0.9,
    'nesterov'      : True,
    'drop_rate'     : 0.25,
    'label_smooth'  : 0.1,
    'drop_path'     : 0.1
}

In [32]:
class SubjectDataset(torch.utils.data.Dataset):

    def __init__(self, root, subject = 1, mode = 'train'):
        
        self.eegs, self.results = [], []
        for experiment in ["Exp1","Exp2"]:
            dir = root + '/' + experiment
            for session in os.listdir(dir):
                should_load = f'Subj{subject}_' in session
                if mode == 'train':
                    should_load = not should_load
                if should_load:
                    # Load a single mat file
                    mat = scipy.io.loadmat(dir+'/'+session)
                    # Get raw EEG data in one session
                    session_eeg = mat['output_data']   
                    # Get target code in one session
                    exp_states = dict()
                    for _ in mat['Experimental_states']:
                        for wrapper in _:
                            fieldnames = wrapper.dtype.names
                            for idx, val in enumerate(wrapper):
                                exp_states[f'{fieldnames[idx]}'] = val.flatten()
                    session_targetCode = exp_states['TargetCode']-1
                    '''
                    TODO: The different effective length of 3 sessions will cause problems for data loader.
                    --> for now, we use a fix number 100 as the effective duration
                    --> If this impacts the model performance, we can try pad_pack_sequence
                    '''
                    
                    # Remove useless data
                    trial_start_index = np.nonzero(np.diff(session_targetCode))[0][::2]+1
                    trial_start_end_index = np.nonzero(np.diff(session_targetCode))[0]+1

                    # Split continuous eeg data to trials
                    eeg_split_trials = np.split(session_eeg,trial_start_end_index)[1::2]
                    
                    # Motor imgination starts 2s after the target shows up. Plus 0.1s for reactions time
                    effective_start = 210
                    
                    # The target freezes for 1s after hitting the target, signals in this 1s is useless.
                    # Use the min duration time - 1s as the effective end time. 
                    # Different sessions have different min duration time
                    # min_trial_duration = np.amin(np.diff(trial_start_end_index)[::2])
                    # effective_end = min_trial_duration - 100
                    
                    # Update: for now, we use a fix number 100 as the effective duration
                    effective_end = effective_start + 100
                    
                    effective_eeg = []
                    for eeg in eeg_split_trials:
                        # Exponential standardization
                        eeg_standardized = np.zeros_like(eeg[effective_start:effective_end,:])
                        mean = 0
                        var = 0
                        alpha = 0.999
                        for t in range(eeg[effective_start:effective_end,:].shape[0]):
                            mean = alpha * mean + (1 - alpha) * eeg[effective_start:effective_end,:][t]
                            var = alpha * var + (1 - alpha) * (eeg[effective_start:effective_end,:][t] - mean) ** 2
                            eeg_standardized[t] = (eeg[effective_start:effective_end,:][t] - mean) / np.sqrt(var + 1e-8)
                        effective_eeg.append(eeg_standardized)
                    
                    # Remove useless target code
                    session_targetCode = session_targetCode[trial_start_index]
                    
                    # Add the session data into the array
                    self.eegs += (effective_eeg)
                    self.results.append(session_targetCode)
        
        # Concatenate results
        self.results = np.concatenate(self.results)    
        self.length = len(self.results)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        eegs = np.array(self.eegs[ind]) # extra [] to make an additional dimension for channel #REMOVED
        eegs = torch.FloatTensor(eegs)
        results = torch.tensor(self.results[ind])
        return eegs, results

    '''
    TODO: Discuss if we need batch-wise operations.
    '''
    # def collate_fn(batch):
    #     return batch

In [249]:
del train_dataset, test_dataset, train_loader, test_loader

In [250]:
torch.cuda.empty_cache()
gc.collect()

0

In [251]:
# Test of dataset and data loader
root = '/content/bci_data'
train_dataset = SubjectDataset(root, subject=1, mode = 'train')
train_loader = torch.utils.data.DataLoader(
    dataset     = train_dataset, 
    num_workers = 2,
    batch_size  = config['batch_size'], 
    pin_memory  = True,
    shuffle     = True,
    # collate_fn  = AudioDataset.collate_fn
)

test_dataset = SubjectDataset(root, subject=1, mode = 'test')
test_loader = torch.utils.data.DataLoader(
    dataset     = test_dataset, 
    num_workers = 2,
    batch_size  = config['batch_size'], 
    pin_memory  = True,
    shuffle     = False,
    # collate_fn  = AudioDataset.collate_fn
)

In [252]:
print("No. of data         : ", train_dataset.__len__())
print("No. of test data    : ", test_dataset.__len__())
print("Shape of data       : ", train_dataset[0][0].shape)

print("Train batches       : ", train_loader.__len__())
print("Test batches        : ", test_loader.__len__())

# Test of dataset and data loader
for x, y in train_loader:
    print(x.shape, y.shape)
    xt, yt = x, y
    break
print(xt.shape)

No. of data         :  9945
No. of test data    :  375
Shape of data       :  torch.Size([100, 62])
Train batches       :  311
Test batches        :  12
torch.Size([32, 100, 62]) torch.Size([32])
torch.Size([32, 100, 62])


# Model:CustomNet (Conv1d embedder - Transformer Encoder - Classification Head)

In [7]:
class CustomNet(nn.Module):
    def __init__(self, in_channels=62, hid_dim=6, n_classes=2, n_layers=5,dropout=0.5):
        super().__init__()
        self.embedder = nn.Sequential(#PermuteBlock(),
            nn.Linear(in_channels, in_channels*2),
            nn.LayerNorm(in_channels*2), torch.nn.GELU(),
            nn.Linear(in_channels*2, in_channels*4),
            nn.LayerNorm(in_channels*4), torch.nn.GELU(),
           )#PermuteBlock())

        self.rnn = nn.LSTM(in_channels*4, hid_dim, num_layers=n_layers, batch_first=True,
                           dropout=dropout)
        self.class_head = nn.Sequential(
            nn.Linear(hid_dim, n_classes),
        )
        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight, gain=1)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.embedder(x)
        x, _ = self.rnn(x)
        x = self.class_head(x)
        return x[:,-1,:]

In [253]:
model = CustomNet(in_channels=62, hid_dim=4, n_classes=2,n_layers=4, dropout=0.45).to(DEVICE)
print(model)
# xt = xt.cuda()
# summary(model, xt)

CustomNet(
  (embedder): Sequential(
    (0): Linear(in_features=62, out_features=124, bias=True)
    (1): LayerNorm((124,), eps=1e-05, elementwise_affine=True)
    (2): GELU(approximate='none')
    (3): Linear(in_features=124, out_features=248, bias=True)
    (4): LayerNorm((248,), eps=1e-05, elementwise_affine=True)
    (5): GELU(approximate='none')
  )
  (rnn): LSTM(248, 4, num_layers=4, batch_first=True, dropout=0.45)
  (class_head): Sequential(
    (0): Linear(in_features=4, out_features=2, bias=True)
  )
)


In [254]:
criterion = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=config['lr'], momentum=config['momentum'], weight_decay=config['weight_decay'])
optimizer = torch.optim.AdamW(model.parameters(), lr=config['lr']) #, weight_decay=0.001)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(patience = 5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
         optimizer=optimizer,
         T_max = config['epochs'],
         eta_min=1e-6,
         verbose=True,
 )
scaler = torch.cuda.amp.GradScaler()

Adjusting learning rate of group 0 to 5.0000e-04.


In [10]:
def train(model, dataloader, optimizer, criterion):
    
    model.train()

    # Progress Bar 
    batch_bar   = tqdm(total=len(dataloader), dynamic_ncols=True, leave=False, position=0, desc='Train', ncols=5) 

    num_correct = 0
    total_loss  = 0

    for i, (x, y) in enumerate(dataloader):
        
        optimizer.zero_grad() # Zero gradients

        x, y = x.to(DEVICE), y.to(DEVICE)
        
        with torch.cuda.amp.autocast(): # This implements mixed precision. Thats it! 
            outputs = model(x)
            loss    = criterion(outputs, y)

        # Update no. of correct predictions & loss as we iterate
        num_correct     += int((torch.argmax(outputs, axis=1) == y).sum())
        total_loss      += float(loss.item())

        # tqdm lets you add some details so you can monitor training as you train.
        batch_bar.set_postfix(
            acc         = "{:.04f}%".format(100 * num_correct / (config['batch_size']*(i + 1))),
            loss        = "{:.04f}".format(float(total_loss / (i + 1))),
            num_correct = num_correct,
            lr          = "{:.04f}".format(float(optimizer.param_groups[0]['lr']))
        )
        
        loss.backward()
        optimizer.step()
        # scaler.scale(loss).backward() # This is a replacement for loss.backward()
        # scaler.step(optimizer) # This is a replacement for optimizer.step()
        # scaler.update() 

        # TODO? Depending on your choice of scheduler,
        # You may want to call some schdulers inside the train function. What are these?
      
        batch_bar.update() # Update tqdm bar

        del x, y, outputs, loss
        torch.cuda.empty_cache()

    batch_bar.close() # You need this to close the tqdm bar

    acc         = 100 * num_correct / (config['batch_size']* len(dataloader))
    total_loss  = float(total_loss / len(dataloader))

    return acc, total_loss

In [11]:
def validate(model, dataloader, criterion):
  
    model.eval()
    batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True, position=0, leave=False, desc='Val', ncols=5)

    num_correct = 0.0
    total_loss = 0.0

    for i, (x, y) in enumerate(dataloader):
        
        # Move images to device
        x, y = x.to(DEVICE), y.to(DEVICE)
        
        # Get model outputs
        with torch.inference_mode():
            outputs = model(x)
            loss = criterion(outputs, y)

        num_correct += int((torch.argmax(outputs, axis=1) == y).sum())
        total_loss += float(loss.item())

        batch_bar.set_postfix(
            acc="{:.04f}%".format(100 * num_correct / (config['batch_size']*(i + 1))),
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            num_correct=num_correct)

        batch_bar.update()
        del x, y, outputs, loss
        torch.cuda.empty_cache()
        
    batch_bar.close()
    acc = 100 * num_correct / (config['batch_size']* len(dataloader))
    total_loss = float(total_loss / len(dataloader))
    return acc, total_loss

In [255]:
best_valacc = 0.0

val_list = []

for epoch in range(config['epochs']):
    curr_lr = float(optimizer.param_groups[0]['lr'])
    train_acc, train_loss = train(model, train_loader, optimizer, criterion)
    
    print("\nEpoch {}/{}: \nTrain Acc {:.04f}%\t Train Loss {:.04f}\t Learning Rate {:.04f}".format(
        epoch + 1,
        config['epochs'],
        train_acc,
        train_loss,
        curr_lr))
    
    val_acc, val_loss = validate(model, test_loader, criterion)
    print("Val Acc {:.04f}%\t Val Loss {:.04f}".format(val_acc, val_loss))

    val_list.append(val_acc)
    
    # wandb.log({"train_loss":train_loss, 'train_Acc': train_acc, 'validation_Acc':val_acc, 
    #            'validation_loss': val_loss, "learning_Rate": curr_lr})
    
    # If you are using a scheduler in your train function within your iteration loop, you may want to log
    # your learning rate differently
    # scheduler.step(val_loss)
    scheduler.step()

    #Save model in drive location if val_acc is better than best recorded val_acc
    # if val_acc >= best_valacc:
    #   #path = os.path.join(root, model_directory, 'checkpoint' + '.pth')
    #   print("Saving model")
    #   torch.save({'model_state_dict':arcfacemodel.model.state_dict(),
    #               'optimizer_state_dict':optimizer.state_dict(),
    #               'scheduler_state_dict':scheduler.state_dict(),
    #               'val_acc': val_acc, 
    #               'epoch': epoch}, './checkpoint.pth')
    #   best_valacc = val_acc
    #   wandb.save('checkpoint.pth')
      # You may find it interesting to exlplore Wandb Artifcats to version your models
# run.finish()


Epoch 1/20: 
Train Acc 69.4634%	 Train Loss 0.6452	 Learning Rate 0.0005


Val Acc 72.6562%	 Val Loss 0.5710
Adjusting learning rate of group 0 to 4.9693e-04.



Epoch 2/20: 
Train Acc 73.3923%	 Train Loss 0.5698	 Learning Rate 0.0005


Val Acc 71.8750%	 Val Loss 0.5497
Adjusting learning rate of group 0 to 4.8779e-04.



Epoch 3/20: 
Train Acc 73.7842%	 Train Loss 0.5580	 Learning Rate 0.0005


...

Val Acc 75.0000%	 Val Loss 0.5058
Adjusting learning rate of group 0 to 1.3211e-05.



Epoch 19/20: 
Train Acc 79.0394%	 Train Loss 0.4790	 Learning Rate 0.0000


Val Acc 74.7396%	 Val Loss 0.5092
Adjusting learning rate of group 0 to 4.0718e-06.



Epoch 20/20: 
Train Acc 78.8284%	 Train Loss 0.4800	 Learning Rate 0.0000


Val Acc 74.4792%	 Val Loss 0.5091
Adjusting learning rate of group 0 to 1.0000e-06.


In [256]:
print(val_list)

[72.65625, 71.875, 71.09375, 71.09375, 73.17708333333333, 76.82291666666667, 77.34375, 75.26041666666667, 75.0, 76.04166666666667, 75.78125, 76.82291666666667, 76.30208333333333, 75.52083333333333, 75.26041666666667, 75.0, 74.47916666666667, 75.0, 74.73958333333333, 74.47916666666667]
